In [24]:
#必要なライブラリをインポート
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import test_func as tf
%matplotlib inline

In [25]:
files = glob.glob("train/*.csv") #拡張子.csvのファイル名を全て抽出
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0)) #appendでファイル名をリストに追加
df = pd.concat(data_list) #抽出したファイルを結合

/Users/shota/.pyenv/versions/3.9.4/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
df = tf.data_pre(df)
model = tf.model_lgb(df)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017549 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11638
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 17


/Users/shota/.pyenv/versions/3.9.4/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/shota/.pyenv/versions/3.9.4/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 7.217054
[1]	valid_0's l1: 0.253354
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.236846
[3]	valid_0's l1: 0.222411
[4]	valid_0's l1: 0.209679
[5]	valid_0's l1: 0.198353
[6]	valid_0's l1: 0.188458
[7]	valid_0's l1: 0.179497
[8]	valid_0's l1: 0.171626
[9]	valid_0's l1: 0.164552
[10]	valid_0's l1: 0.158269
[11]	valid_0's l1: 0.152753
[12]	valid_0's l1: 0.148
[13]	valid_0's l1: 0.143241
[14]	valid_0's l1: 0.139211
[15]	valid_0's l1: 0.135694
[16]	valid_0's l1: 0.132447
[17]	valid_0's l1: 0.129606
[18]	valid_0's l1: 0.126912
[19]	valid_0's l1: 0.12464
[20]	valid_0's l1: 0.122603
[21]	valid_0's l1: 0.12068
[22]	valid_0's l1: 0.118644
[23]	valid_0's l1: 0.116731
[24]	valid_0's l1: 0.115021
[25]	valid_0's l1: 0.113541
[26]	valid_0's l1: 0.112168
[27]	valid_0's l1: 0.110589
[28]	valid_0's l1: 0.109384
[29]	valid_0's l1: 0.108178
[30]	valid_0's l1: 0.106955
[31]	valid_0's l1: 0.106017
[32]	valid_0's l1: 0.104967
[33

In [10]:
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1) #特定のカラムを削除する。

    df = df.drop("市区町村名", axis=1)
    #市区町村コードと同義であるため削除
    df = df.drop("種類", axis=1)
    #df.astype("str").describe()で"種類"のuniqueは１つのみであったため削除

    #"最寄駅：距離（分）"に1H?1H30のような要素があるため変換する。
    dis = {
          "30分?60分": 45,
          "1H?1H30": 75,
          "2H?": 120,
          "1H30?2H": 105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)
    #"2000㎡以上"があるため変換

    #令和,平成,昭和を数値変換
    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    #"年第-四半期"を数値変換
    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".5",
        "年第３四半期": ".75",
        "年第４四半期": ".99",
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k,j)
        year_list[i] = year_rep

    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[col] = df[col].astype("category")
    
    
    return df



In [11]:
df = data_pre(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637351 entries, 40108844 to 12009989
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       637351 non-null  int64   
 1   都道府県名         637351 non-null  category
 2   地区名           637060 non-null  category
 3   最寄駅：名称        634732 non-null  category
 4   最寄駅：距離（分）     614306 non-null  float64 
 5   間取り           615609 non-null  category
 6   面積（㎡）         637351 non-null  float64 
 7   建築年           619117 non-null  float64 
 8   建物の構造         623423 non-null  category
 9   用途            591214 non-null  category
 10  今後の利用目的       275091 non-null  category
 11  都市計画          618786 non-null  category
 12  建ぺい率（％）       614848 non-null  float64 
 13  容積率（％）        614848 non-null  float64 
 14  取引時点          637351 non-null  float64 
 15  改装            580431 non-null  category
 16  取引の事情等        18275 non-null   category
 17  取引価格（総額）_log  637351

# ここからlightGBMを使う

In [12]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val =train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11687
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 17


/Users/shota/.pyenv/versions/3.9.4/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/shota/.pyenv/versions/3.9.4/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 7.217308
[1]	valid_0's l1: 0.253279
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.236818
[3]	valid_0's l1: 0.222448
[4]	valid_0's l1: 0.20963
[5]	valid_0's l1: 0.198214
[6]	valid_0's l1: 0.188148
[7]	valid_0's l1: 0.179496
[8]	valid_0's l1: 0.171766
[9]	valid_0's l1: 0.164569
[10]	valid_0's l1: 0.158112
[11]	valid_0's l1: 0.152603
[12]	valid_0's l1: 0.14766
[13]	valid_0's l1: 0.143252
[14]	valid_0's l1: 0.139247
[15]	valid_0's l1: 0.135583
[16]	valid_0's l1: 0.132274
[17]	valid_0's l1: 0.129456
[18]	valid_0's l1: 0.126949
[19]	valid_0's l1: 0.12444
[20]	valid_0's l1: 0.122234
[21]	valid_0's l1: 0.120295
[22]	valid_0's l1: 0.118028
[23]	valid_0's l1: 0.116324
[24]	valid_0's l1: 0.114685
[25]	valid_0's l1: 0.112963
[26]	valid_0's l1: 0.11156
[27]	valid_0's l1: 0.11017
[28]	valid_0's l1: 0.108784
[29]	valid_0's l1: 0.107632
[30]	valid_0's l1: 0.106554
[31]	valid_0's l1: 0.105677
[32]	valid_0's l1: 0.104879
[33

In [13]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07625839584131572

In [14]:
df_test = pd.read_csv("test.csv", index_col=0)
df_test = data_pre(df_test)

In [16]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")

In [20]:
#特徴量重要度を確認する。
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,8634
最寄駅：名称,6361
建築年,3416
取引時点,3274
面積（㎡）,2120
最寄駅：距離（分）,1953
間取り,817
市区町村コード,783
都道府県名,555
改装,490
